# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on Structured Streaming (Kafka)** </center>
---
**Profesor**: Pablo Camarillo Ramirez

# Install Kafka SERVER

Go to `kafka` folder and run:

`docker compose up -d`

# Create SparkSession
## Install Kafka connector

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (Kafka)") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b9c47a09-b8b5-46e9-ad6c-46aaa9de0576;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.0 in central
	found org.apache.kafka#kafka-clients;3.9.0 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.2.0

# Create a data stream from a Kafka topic

In [2]:
# Create the remote connection
kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "kafka:9093") \
            .option("subscribe", "kafka-spark-example") \
            .load()
kafka_df.printSchema()

# Transform binary data to string
df_input = kafka_df.selectExpr("CAST(value AS STRING)")

# Send transformed data to the Sink
query_a = df_input.writeStream \
            .trigger(processingTime='1 second') \
            .outputMode("append") \
            .format("console") \
            .start()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



25/10/10 00:56:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-107e8b41-e396-4198-a616-4d2b4877b163. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/10 00:56:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+



25/10/10 00:56:14 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000} milliseconds, but spent 3091 milliseconds
25/10/10 00:57:25 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000} milliseconds, but spent 2786 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
|value|
+-----+
|  wow|
+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+
|value|
+-----+
|hola2|
+-----+



In [3]:
query_a.stop()
sc.stop()

25/10/10 01:01:03 WARN DAGScheduler: Failed to cancel job group 40a5c0d8-194d-428e-8c0e-f286f2567dff. Cannot find active jobs for it.
25/10/10 01:01:03 WARN DAGScheduler: Failed to cancel job group 40a5c0d8-194d-428e-8c0e-f286f2567dff. Cannot find active jobs for it.


## Add a Streaming listener

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pcamarillor.streaming_listener import MyListener

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (Kafka)") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Add custom listener
spark.streams.addListener(MyListener())

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cc7f0d3c-c6a4-4faf-9078-2ce3dcbb1fde;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.0 in central
	found org.apache.kafka#kafka-clients;3.9.0 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.2.0

Query started: a50aa8f6-0796-4466-8304-88a80c9b25e6
Query made progress: {
  "id" : "a50aa8f6-0796-4466-8304-88a80c9b25e6",
  "runId" : "389240a8-931f-4eaa-87a0-45c041409f87",
  "name" : null,
  "timestamp" : "2025-10-10T01:29:59.861Z",
  "batchId" : 0,
  "batchDuration" : 4876,
  "numInputRows" : 0,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "addBatch" : 3150,
    "commitOffsets" : 158,
    "getBatch" : 60,
    "latestOffset" : 733,
    "queryPlanning" : 673,
    "triggerExecution" : 4871,
    "walCommit" : 76
  },
  "stateOperators" : [ {
    "operatorName" : "stateStoreSave",
    "numRowsTotal" : 0,
    "numRowsUpdated" : 0,
    "allUpdatesTimeMs" : 50,
    "numRowsRemoved" : 0,
    "allRemovalsTimeMs" : 0,
    "commitTimeMs" : 217,
    "memoryUsedBytes" : 1200,
    "numRowsDroppedByWatermark" : 0,
    "numShufflePartitions" : 5,
    "numStateStoreInstances" : 5,
    "customMetrics" : {
      "loadedMapCacheHitCount" : 0,
      "loadedMapC

In [3]:
# Create the remote connection
from pyspark.sql.functions import explode, split
kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "kafka:9093") \
            .option("subscribe", "kafka-spark-example") \
            .load()


# Transform binary data to string
df_input = kafka_df.selectExpr("CAST(value AS STRING)")
df_input = df_input.select(explode(split(df_input.value, " ")).alias("word"))
df_input = df_input.groupBy("word").count()

# Send transformed data to the Sink
query_b = df_input.writeStream \
            .trigger(processingTime='3 second') \
            .outputMode("complete") \
            .format("console") \
            .start()


query_b.awaitTermination(300)

25/10/10 01:29:59 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9b50376e-6272-4525-bd49-654b01a40de1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/10 01:29:59 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



25/10/10 01:30:04 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000} milliseconds, but spent 4907 milliseconds
                                                                                

-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|      mi|    1|
|  nombre|    1|
|      es|    1|
|Edudardo|    1|
|    hola|    1|
+--------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|    otra|    1|
|     vez|    1|
|      mi|    1|
|  nombre|    1|
|      es|    1|
|Edudardo|    1|
|    hola|    2|
+--------+-----+



False

In [ ]:
query_b.stop()
sc.stop()